## **Long Short_Term Memory**


In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error as mae
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from keras.callbacks import EarlyStopping
from scipy.stats import pearsonr

In [ ]:
# Đọc dữ liệu từ file excel
data9 = pd.read_excel('drive/MyDrive/data_match9.xlsx', engine='openpyxl')
data10 = pd.read_excel('drive/MyDrive/data_match10.xlsx', engine='openpyxl')


data9.dropna(inplace=True)
data10.dropna(inplace=True)



In [ ]:
#Xử lý data

X_9 = data9[['B04B', 'B05B', 'B06B', 'B09B', 'B10B', 'B11B', 'B12B', 'B14B', 'B16B',
                 'I2B', 'I4B', 'IRB', 'VSB', 'WVB', 'CAPE', 'TCC', 'TCW', 'TCWV']].values
y_9 = data9['value'].values
# print(X_9[:, -1])
X_10 = data10[['B04B', 'B05B', 'B06B', 'B09B', 'B10B', 'B11B', 'B12B', 'B14B', 'B16B',
                 'I2B', 'I4B', 'IRB', 'VSB', 'WVB', 'CAPE', 'TCC', 'TCW', 'TCWV']].values
y_10 = data10['value'].values

X = np.concatenate((X_9, X_10), axis=0)
y = np.concatenate((y_9, y_10), axis=0)

# print(data.shape)

# Scale data range
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

y = scaler.fit_transform(y.reshape(-1, 1))

# X = np.concatenate((X_scale, y), axis = 1)
print(X.shape)
print(y.shape)
# # Divide train/test set: 80/20
size = int(len(X) * 0.8)
X_train_data, X_test_data = X[:size], X[size:]
y_train_data, y_test_data = y[:size], y[size:]
print(y_train_data.shape)
# def create_dataset(datasetX, datasetY, look_back):
#     X_data, Y_data = [], []
#     for i in range(look_back, len(datasetX)):
#         X_data.append(datasetX[i-look_back:i, :])
#         Y_data.append(datasetY[i, 0])
#     return np.array(X_data), np.array(Y_data)
def create_dataset(datasetX, datasetY, look_back):
    X_data, Y_data = [], []
    for i in range(len(datasetX)-look_back):
        X_data.append(datasetX[i:(i+look_back), :])
        Y_data.append(datasetY[i + look_back, 0])
    return np.array(X_data), np.array(Y_data)

look_back = 5

X_train, y_train = create_dataset(X_train_data, y_train_data,look_back)
X_test, y_test = create_dataset(X_test_data, y_test_data, look_back)

#Reshape to [samples, time steps, features]
# X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(161936, 18)
(161936, 1)
(129548, 1)
(129543, 5, 18)
(129543,)
(32383, 5, 18)
(32383,)


In [ ]:
#Reshape input
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:

# Init model
model = Sequential()
model.add(LSTM(50, activation = 'relu', return_sequences=True, input_shape=(look_back, 18)))
# model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')


model.fit(X_train, y_train, epochs=30, batch_size=64, 
                    validation_data=(X_test, y_test), shuffle=False, verbose = 1)


Epoch 1/30
2025/2025 [==============================] - 35s 15ms/step - loss: 6.7456e-04 - val_loss: 8.6258e-04
Epoch 2/30
2025/2025 [==============================] - 31s 15ms/step - loss: 6.3382e-04 - val_loss: 8.5430e-04
Epoch 3/30
2025/2025 [==============================] - 29s 15ms/step - loss: 6.2323e-04 - val_loss: 8.4934e-04
Epoch 4/30
2025/2025 [==============================] - 29s 14ms/step - loss: 6.1954e-04 - val_loss: 8.4437e-04
Epoch 5/30
2025/2025 [==============================] - 31s 15ms/step - loss: 6.1576e-04 - val_loss: 8.4505e-04
Epoch 6/30
2025/2025 [==============================] - 30s 15ms/step - loss: 6.1307e-04 - val_loss: 8.4458e-04
Epoch 7/30
2025/2025 [==============================] - 33s 16ms/step - loss: 6.1363e-04 - val_loss: 8.4306e-04
Epoch 8/30
2025/2025 [==============================] - 30s 15ms/step - loss: 6.0942e-04 - val_loss: 8.4550e-04
Epoch 9/30
2025/2025 [==============================] - 32s 16ms/step - loss: 6.1013e-04 - val_loss: 8.4

In [ ]:
#Predict Data
y_predict_train = model.predict(X_train)
print(y_predict_train.shape)

#Scale Data
y_train = scaler.inverse_transform(y_train.reshape(-1,1))
y_predict_train = scaler.inverse_transform(y_predict_train.reshape(-1, 1))
# print(y_train)

y_predict_test = model.predict(X_test)
y_test = scaler.inverse_transform(y_test.reshape(-1,1))
y_predict_test = scaler.inverse_transform(y_predict_test.reshape(-1, 1))


4049/4049 [==============================] - 16s 4ms/step
(129543, 1)
1012/1012 [==============================] - 3s 3ms/step


In [ ]:
#Train performance 
print('Training...')
print('Train R2 score: ', r2_score(y_train, y_predict_train))
# print('Train MSE: ', train_mse)
# print('Train RMSE: ', np.sqrt(train_mse))
mse = mean_squared_error(y_train, y_predict_train)
print('Train MSE: ', mse)
print('Train RMSE: ', np.sqrt(mse))
corr_train, p_value_train = pearsonr(y_train.reshape(-1), y_predict_train.reshape(-1))
print('Train MAE:', mae(y_predict_train, y_train))
print('Train Pearson r:', corr_train, p_value_train)

print('Testing...')
#Test performance
print('Test R2 score: ', r2_score(y_test, y_predict_test))
# print('Test MSE: ', test_mse)
# print('Test RMSE: ', np.sqrt(test_mse))
mse = mean_squared_error(y_test, y_predict_test)
print('Test MSE: ', mse)
print('Test RMSE: ', np.sqrt(mse))
corr_test, p_value_test = pearsonr(y_test.reshape(-1), y_predict_test.reshape(-1))
print('Train MAE:', mae(y_predict_test, y_test))
print('Train Pearson r:', corr_test, p_value_test)


Training...
Train R2 score:  0.17061831895836566
Train MSE:  4.3842900012750174
Train RMSE:  2.0938696237528776
Train MAE: 0.5297567665251163
Train Pearson r: 0.4138775158075171 0.0
Testing...
Test R2 score:  0.06242467276142638
Test MSE:  6.394907674051855
Test RMSE:  2.528815468564651
Train MAE: 0.517909125809961
Train Pearson r: 0.2867476887425642 0.0
